In [1]:
!pip install transformers

     |████████████████████████████████| 2.3MB 33.3MB/s 
     |████████████████████████████████| 901kB 49.7MB/s 
     |████████████████████████████████| 3.3MB 40.3MB/s 


In [2]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Data Science Projects/Toxic Comment Classification/train.csv')

In [4]:
df=df[:2000]

In [5]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
classes=['toxic',	'severe_toxic',	'obscene',	'threat',	'insult',	'identity_hate']

In [7]:
# Removing the sentences which have length greater than 50

lst=[]
for i,val in enumerate(df.comment_text):
  if len(val.split())<50:
    lst.append(i)

df=df[df.index.isin(lst)]

In [8]:
# Importing pre trained BERT model and tokenizer
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
# Tokenization
# The tokenizer tokenizes the sentence into tokens(words), adds special tokens i.e. [CLS] and [SEP] at the start and end,
# and then substitutes tokens with their id's.

tokenized = df['comment_text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))  # we can use map instead of apply

In [10]:
# Padding

# After tokenization, tokenized becomes a list of sentences and each sentence is represented as a list of tokens.
# We may pad each sentence so that the length of every sentence becomes same and we can give BERT a 2-d array instead of list of lists.

maxm = max([len(i) for i in tokenized.values])

padded = np.array([i + [0]*(maxm-len(i)) for i in tokenized.values])

In [11]:
# The dataset is now in padded variable and we can check then shape of 2-D matrix using
padded.shape

(1207, 124)

In [12]:
# Masking

# If we directly send padded to BERT, that would slightly confuse it. We need to create another variable to tell it to 
# ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

attention_mask = np.where(padded !=0, 1, 0)

attention_mask.shape

(1207, 124)

In [13]:
# Now create input tensor out of the input matrix and attention mask and pass it to the BERT model
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
  last_hidden_states = model(input_ids, attention_mask)

# After this step last_hidden_states hold the output of BERT model. 
# It is a tuple with shape(number of input examples, max sequence length, number of hidden units in the BERT model)

In [14]:
# since this is classification model, we are only interested in the output of [CLS] token.

features = last_hidden_states[0][:,0,:].numpy()

labels = df.loc[:,'toxic':'identity_hate']

In [15]:
# Train/test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features,labels)

In [17]:
!pip install scikit-multilearn

     |████████████████████████████████| 92kB 4.7MB/s 


In [18]:
# Classifier Chain method for Multi Label Classification

from skmultilearn.problem_transform import ClassifierChain
from sklearn.naive_bayes import GaussianNB

# initialize classifier chains multi-label classifier
# with a gaussian naive bayes base classifier
classifier = ClassifierChain(GaussianNB())

# train
classifier.fit(X_train, y_train)

# predict
predictions = classifier.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,predictions)

0.8013245033112583

## Predicting the labels for a given text

In [19]:
text=' Whoever put a notices on my page. I will kill u'
tokenized_text=tokenizer.encode(text,add_special_tokens= True)
padded_text = np.array(tokenized_text+[0]*(maxm-len(tokenized_text)))
attention_mask_text = np.where(padded_text !=0, 1, 0)

input_ids = torch.tensor([padded_text])
attention_mask = torch.tensor([attention_mask_text])

with torch.no_grad():
  last_hidden_states = model(input_ids, attention_mask)

for class_name, label in zip(classes, classifier.predict(last_hidden_states[0][:,0,:].numpy()).toarray()[0]):
  print(class_name+" : "+str(label), sep='\n')

toxic : 1.0
severe_toxic : 0.0
obscene : 0.0
threat : 1.0
insult : 1.0
identity_hate : 1.0
